In [1]:
import os
os.chdir("..")
pwd = %pwd
pwd[12:]

'PJATK/semestr4/ADD/Covid19_Prediction'

In [2]:
import pandas as pd

In [3]:
import json
with open("data/covid_data.json") as f:
    d = json.load(f)

In [4]:
import numpy as np
import os
import typing

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
def preprocess_data(data: dict):
    df = pd.DataFrame(data["records"])
    df = df[df["countryterritoryCode"] == "POL"]
    df['dateRep'] = pd.to_datetime(df['dateRep'], format="%d/%m/%Y")
    df = df.sort_values("dateRep")
    cleaned_data = df.drop(["countryterritoryCode", "continentExp", "geoId", "countriesAndTerritories", "dateRep", "day", "month", "year", "popData2020", "deaths"], axis=1)
    cleaned_data['cases_next_day'] = df['cases'].shift(-1)
    x = cleaned_data.drop(["cases_next_day"],  axis=1).values[:-1]
    y = cleaned_data["cases_next_day"].values[:-1]
    return train_test_split(x, y, test_size=0.3, shuffle=False)

In [7]:
x_train, x_val, y_train, y_val = preprocess_data(d)

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import layers

2023-05-30 20:38:20.420182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
SEQUENCE_LENGTH=5

In [12]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=SEQUENCE_LENGTH
)

2023-05-30 20:38:30.695125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=14,
#     sampling_rate=step,
#     batch_size=batch_size,
)


for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Input value:", inputs.numpy()[0][0][0])
print("Input value:", inputs.numpy()[0][1][0])
# print("Input value:", inputs.numpy()[0][2][0])

print("Target shape:", targets.numpy().shape)
print("Target value:", targets.numpy()[0])

Input shape: (128, 5, 1)
Input value: 1.0
Input value: 0.0
Target shape: (128,)
Target value: 0.0


In [14]:
model = tf.keras.Sequential([
    keras.layers.LSTM(32),
    tf.keras.layers.Dense(units=16, activation="relu"),
    tf.keras.layers.Dense(units=1)

    
])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=6,
                                                    mode='min')

model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.fit(dataset_train, 
                     epochs=4000,
                     batch_size=300,
                     validation_data=dataset_val,
                     callbacks=[early_stopping])

Epoch 1/4000
6/6 [==============================] - 3s 122ms/step - loss: 106092016.0000 - mean_absolute_error: 6177.0811 - val_loss: 194749152.0000 - val_mean_absolute_error: 7128.7822
Epoch 2/4000
6/6 [==============================] - 0s 27ms/step - loss: 106089976.0000 - mean_absolute_error: 6176.9111 - val_loss: 194747248.0000 - val_mean_absolute_error: 7128.6396
Epoch 3/4000
6/6 [==============================] - 0s 26ms/step - loss: 106088304.0000 - mean_absolute_error: 6176.7695 - val_loss: 194745296.0000 - val_mean_absolute_error: 7128.4941
Epoch 4/4000
6/6 [==============================] - 0s 23ms/step - loss: 106086592.0000 - mean_absolute_error: 6176.6226 - val_loss: 194743312.0000 - val_mean_absolute_error: 7128.3442
Epoch 5/4000
6/6 [==============================] - 0s 23ms/step - loss: 106084864.0000 - mean_absolute_error: 6176.4692 - val_loss: 194741184.0000 - val_mean_absolute_error: 7128.1753
Epoch 6/4000
6/6 [==============================] - 0s 23ms/step - loss: 1

6/6 [==============================] - 0s 23ms/step - loss: 105480616.0000 - mean_absolute_error: 6128.1250 - val_loss: 194021680.0000 - val_mean_absolute_error: 7077.7085
Epoch 46/4000
6/6 [==============================] - 0s 22ms/step - loss: 105447120.0000 - mean_absolute_error: 6125.5156 - val_loss: 193983456.0000 - val_mean_absolute_error: 7075.2266
Epoch 47/4000
6/6 [==============================] - 0s 22ms/step - loss: 105415488.0000 - mean_absolute_error: 6123.1577 - val_loss: 193951904.0000 - val_mean_absolute_error: 7073.0898
Epoch 48/4000
6/6 [==============================] - 0s 21ms/step - loss: 105387984.0000 - mean_absolute_error: 6120.9756 - val_loss: 193919872.0000 - val_mean_absolute_error: 7070.9263
Epoch 49/4000
6/6 [==============================] - 0s 22ms/step - loss: 105360192.0000 - mean_absolute_error: 6118.7539 - val_loss: 193887472.0000 - val_mean_absolute_error: 7068.7290
Epoch 50/4000
6/6 [==============================] - 0s 21ms/step - loss: 105331624.

6/6 [==============================] - 0s 24ms/step - loss: 103930560.0000 - mean_absolute_error: 6011.8926 - val_loss: 192225248.0000 - val_mean_absolute_error: 6963.6567
Epoch 90/4000
6/6 [==============================] - 0s 21ms/step - loss: 103887792.0000 - mean_absolute_error: 6008.9507 - val_loss: 192175440.0000 - val_mean_absolute_error: 6959.6636
Epoch 91/4000
6/6 [==============================] - 0s 22ms/step - loss: 103844768.0000 - mean_absolute_error: 6005.5991 - val_loss: 192125600.0000 - val_mean_absolute_error: 6957.7261
Epoch 92/4000
6/6 [==============================] - 0s 22ms/step - loss: 103801512.0000 - mean_absolute_error: 6003.0815 - val_loss: 192075216.0000 - val_mean_absolute_error: 6953.7920
Epoch 93/4000
6/6 [==============================] - 0s 21ms/step - loss: 103757920.0000 - mean_absolute_error: 5999.4067 - val_loss: 192024640.0000 - val_mean_absolute_error: 6950.7046
Epoch 94/4000
6/6 [==============================] - 0s 22ms/step - loss: 103714088.

Epoch 133/4000
6/6 [==============================] - 0s 21ms/step - loss: 101709184.0000 - mean_absolute_error: 5869.2437 - val_loss: 189645232.0000 - val_mean_absolute_error: 6825.8911
Epoch 134/4000
6/6 [==============================] - 0s 21ms/step - loss: 101652440.0000 - mean_absolute_error: 5866.7407 - val_loss: 189579728.0000 - val_mean_absolute_error: 6823.0972
Epoch 135/4000
6/6 [==============================] - 0s 21ms/step - loss: 101595264.0000 - mean_absolute_error: 5862.3735 - val_loss: 189512912.0000 - val_mean_absolute_error: 6819.2393
Epoch 136/4000
6/6 [==============================] - 0s 22ms/step - loss: 101537992.0000 - mean_absolute_error: 5858.7290 - val_loss: 189445184.0000 - val_mean_absolute_error: 6815.4321
Epoch 137/4000
6/6 [==============================] - 0s 21ms/step - loss: 101480744.0000 - mean_absolute_error: 5858.4136 - val_loss: 189379696.0000 - val_mean_absolute_error: 6812.8657
Epoch 138/4000
6/6 [==============================] - 0s 22ms/ste

Epoch 177/4000
6/6 [==============================] - 0s 23ms/step - loss: 99061816.0000 - mean_absolute_error: 5739.4268 - val_loss: 186474928.0000 - val_mean_absolute_error: 6684.9590
Epoch 178/4000
6/6 [==============================] - 0s 21ms/step - loss: 98904784.0000 - mean_absolute_error: 5725.0693 - val_loss: 186386800.0000 - val_mean_absolute_error: 6678.2007
Epoch 179/4000
6/6 [==============================] - 0s 22ms/step - loss: 98835800.0000 - mean_absolute_error: 5719.4565 - val_loss: 186308032.0000 - val_mean_absolute_error: 6674.9385
Epoch 180/4000
6/6 [==============================] - 0s 22ms/step - loss: 98767576.0000 - mean_absolute_error: 5713.5620 - val_loss: 186231024.0000 - val_mean_absolute_error: 6671.7954
Epoch 181/4000
6/6 [==============================] - 0s 21ms/step - loss: 98699544.0000 - mean_absolute_error: 5711.5796 - val_loss: 186148976.0000 - val_mean_absolute_error: 6667.3970
Epoch 182/4000
6/6 [==============================] - 0s 22ms/step - l

6/6 [==============================] - 0s 23ms/step - loss: 95674432.0000 - mean_absolute_error: 5567.8813 - val_loss: 182657952.0000 - val_mean_absolute_error: 6540.3540
Epoch 222/4000
6/6 [==============================] - 0s 22ms/step - loss: 95594088.0000 - mean_absolute_error: 5564.7236 - val_loss: 182566816.0000 - val_mean_absolute_error: 6537.5552
Epoch 223/4000
6/6 [==============================] - 0s 21ms/step - loss: 95513888.0000 - mean_absolute_error: 5561.5747 - val_loss: 182473680.0000 - val_mean_absolute_error: 6534.4897
Epoch 224/4000
6/6 [==============================] - 0s 22ms/step - loss: 95433944.0000 - mean_absolute_error: 5562.9912 - val_loss: 182388608.0000 - val_mean_absolute_error: 6536.4624
Epoch 225/4000
6/6 [==============================] - 0s 22ms/step - loss: 95476640.0000 - mean_absolute_error: 5565.1191 - val_loss: 182286000.0000 - val_mean_absolute_error: 6528.8418
Epoch 226/4000
6/6 [==============================] - 0s 22ms/step - loss: 95911072.0

In [15]:
predictions = model.predict(x_val)

10/10 [==============================] - 0s 1ms/step


## evaluation

In [16]:
mean_squared_error(y_val, predictions, squared=False) # squared: If True returns MSE value, if False returns RMSE value. (we want RMSE)

13301.851458589852

In [17]:
r2_score(y_val, predictions)

-0.2718338427769591